In [1]:
from utils import css_from_file
css_from_file('style/style.css')

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier

def load(path):
    df = pd.read_csv(path)
    if "Activity" not in df.columns:
        df["Activity"] = np.nan
    return df.drop("Activity",axis=1), df.Activity
    
X_tr, y_tr = load("data/boehringer/train.csv")
X_te, y_te = load("data/boehringer/test.csv")

print("training data shape", X_tr.shape)
print("testing data shape", X_te.shape)

training data shape (3751, 1776)
testing data shape (2501, 1776)


Stacking
---------------------------

In general stacking is a method of blending the models where you treat the predictions from 1 set of models as features for another model. You can think about comittee of experts. In time you are learning who to listen to.

In previous exercise we used a very simple way to combine models together.
We used a linear combination of predictions

There is an excellent article about ensembling here: http://mlwave.com/kaggle-ensembling-guide/

Another great source of information: https://www.kaggle.com/c/otto-group-product-classification-challenge/forums/t/14335/1st-place-winner-solution-gilberto-titericz-stanislav-semenov - here you can see stacking in action

Exercise
----------------------

1. Read code below and understand what it does?
2. Try to use your classifier from the previous exercise? Does stacking improve a simple average?
3. Try other `mixer` models.
4. After fitting LogisticRegression mixer look at the `coef_` parameter. Which model has the highest weight?

In [15]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from cross_validation import cross_val_apply
from stacked_classifier import StackedClassifier
#from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier

clfs = [
    RandomForestClassifier(n_estimators=300,n_jobs=-1, criterion = 'entropy', max_depth = 50,
                                max_features = 350, random_state = 123),
    GradientBoostingClassifier(),
    ExtraTreesClassifier(n_estimators=100,n_jobs=-1),
    BaggingClassifier(n_estimators=100,n_jobs=-1),
    make_pipeline(StandardScaler(), SVC(probability=True))
]
stacked_classifier = StackedClassifier(estimators=clfs,
                                       mixer=LogisticRegression(),
                                       cv=4,
                                       n_jobs=-1,
                                       probability=True)
oof_predictions = cross_val_apply(stacked_classifier,
                                  X_tr,
                                  y_tr,
                                  cv=4,
                                  n_jobs=1, 
                                  decision_func="predict_proba")
err = log_loss(y_tr, oof_predictions)
print("Your error is", err)

/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/greg/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_a

Your error is 0.443050344697


Double click to see the answers

<div class="spoiler">
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from cross_validation import cross_val_apply
from stacked_classifier import StackedClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

clfs = [
    RandomForestClassifier(n_estimators=100,n_jobs=1),
    GradientBoostingClassifier(),
    ExtraTreesClassifier(n_estimators=100,n_jobs=1),
    XGBClassifier(),
    BaggingClassifier(n_estimators=100),
    make_pipeline(StandardScaler(), SVC(probability=True))
]

stacked_classifier = StackedClassifier(estimators=clfs,
                                       mixer=LogisticRegression(),
                                       cv=4,
                                       n_jobs=1,
                                       probability=True)

oof_predictions = cross_val_apply(stacked_classifier,
                                  X_tr,
                                  y_tr,
                                  cv=4,
                                  n_jobs=-1, 
                                  decision_func="predict_proba")

err = log_loss(y_tr, oof_predictions)
print("Your error is", err)
</div>